In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import torch
import torchvision.transforms as transforms
from torchvision import models
#from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm # Progession bar
import pickle
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [5]:
data_dir = Path.cwd().parent / 'data'
# images_dir = data_dir / 'extracted_images' / 'images'
imager_dir = 'C:/Users/RSCBAL04/Documents/images_1'
metadata_dir = data_dir / 'Data_Entry_2017_v2020.csv'
train_list_path = data_dir / 'train_val_list.txt'
test_list_path = data_dir / 'test_list.txt'


In [6]:
import sys
sys.path.append('../src') 
from model1 import JointLearningModel

In [11]:
# Cargar las imágenes disponibles en el directorio
available_images = set(os.listdir(images_dir))


def filter_and_save_list(list_path, output_path):
    with open(list_path, 'r') as f:
        images = [line.strip() for line in f.readlines()]

    filtered_images = [img for img in images if img in available_images]

    with open(output_path, 'w') as f:
        f.writelines(f"{img}\n" for img in filtered_images)
    return filtered_images

# Filtrar y guardar ambas listas
filtered_train_list = filter_and_save_list(train_list_path, train_list_path)
filtered_test_list = filter_and_save_list(test_list_path, test_list_path)


In [ ]:
# Configuración del cuaderno
import torch
import torch.optim as optim
from model1 import JointLearningModel
from train import train_model
from data_loader import ChestXray8Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),             # Redimensionar a 224x224
    transforms.ToTensor(),                     # Convertir a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalización
])


train_dataset = ChestXray8Dataset(
    img_dir=images_dir, 
    metadata_file=metadata_dir, 
    split_file=train_list_path,
    mode='train',  # Training mode
    transform=transform
)

# Create the DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
# Inicializar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = JointLearningModel(num_clusters=14, embedding_dim=128, pretrained=True).to(device)

# Definir optimizador y función de pérdida
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()

In [ ]:
# Entrenamiento del modelo
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  
    epoch_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        embeddings, cluster_probs = model(images)

        # Clustering requiere logits
        loss = criterion(cluster_probs, labels.argmax(dim=1))  # one-hot

        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")